# How to read this document

This document is divided in two parts : 
- The first part is for the intermediate level tutorial. It is designed for a first contact with hyperspy.
- The second part is for the expert level tutorial. It is designed for learning the advanced features of hyperspy.

Not everything is intended to be read the from the first time. Optional comments along the document are marked by emojis : 
- ⭐ This is a tip, describing more in depth the inner workings of hyperspy.
- 🏠 It's an optional comment. Something that will not be used here but that you can use back to your lab.


# I. Introduction to Hyperspy

Hyperspy is a tool for the visualisation and manipulation of high-dimensional data. The main features of Hyperspy are the following:
- The Signal1D and Signal2D objects 
- Domain-specific features for EELS, EDX, etc ...
- Model fitting
- Machine learning

In this practical we will give an introduction to the Signal objects blablabla

# 🏠 I. Installing Hyperspy with anaconda

This section is intended to help you to install hyperspy on your own computer in your own lab. **You can skip this part for the practical.**

Anaconda is python package manager dedicated to science. Using Anaconda it is possible to create virtual environments. You can then manage different virtual environments to isolate different distributions of python packages. This way you can keep stable python distributions safe from unstable ones.

We describe here the few steps required to create an environment with Hyperspy.

### Install Anaconda

Depending on your operating system install anaconda : https://www.anaconda.com/products/individual

Then, launch the Anaconda Prompt, a console should appear.

### Create a virtual environment

Run the command : ```conda create --name myenv``` (change "myenv" by any name you like.)

Your virtual environment is created but you're still in the default one called "base". You need to activate it by typing: ```conda activate myenv```

### Installing packages

When installing a package there are two solutions: 
1. Either it is part of the already available libraries 
2. Either you can access it from anaconda forge

(1). Run the command : ```conda install package-name``` with for example: ```conda install scikit-learn```

(2). The available python packages are listed on : https://anaconda.org/ . After searching for the library you want, you can click on its name in bold. There will be a list of commands to type and execute to install the selected package. For Hyperspy it is: ```conda install -c conda-forge hyperspy```

### installing ipython

The next few steps will enable you to run the virtual environment you created in ipython notebooks. In the Anaconda prompt run the following commands :
- ```conda install jupyter```
- ```conda install ipykernels```
- ```python -m ipykernel install --user --name myenv```

Now you can start the ipython notebook environment using: ```jupyter notebook```. You will be able to create ipython notebooks (top right corner) with the virtual environments you want. All the packages installed (and their versions) will be specific to that environment. 

# I. Import Hyperspy

⭐ We choose first a graphical backend that manage interactivity (```%matplotlib qt```).

In python the libraries have to be imported to use their functions. For clarity, hyperspy will be imported with the name "hs". 

In [1]:
%matplotlib qt
import hyperspy.api as hs

# I. Loading data

Hyperspy is able to load and manage data from many different open-source and proprietary formats, such as: 
- dm3, dm4
- emd, bcf
- tiff, jpg, etc...

For an exhaustive list : http://hyperspy.org/hyperspy-doc/current/user_guide/io.html#supported-formats

⭐ Without arguments ```hs.load()```  makes a file explorer window pop up. You can also load a list of files.

In [2]:
spim = hs.load("sim_spim_sigma5.hspy")

# I. Data structure and visualisation

The loaded data have a signal type, such as : `EELSSpectrum`, `EDSTEMSpectrum` or `EDSSEMSpectrum`.

Optionnally the data can have a title. 

The main feature of hyperspy signals is the dimension. The dimensions are either navigation or signal. For a spectrum image there are two navigation dimensions X and Y and one signal dimension Z. It is noted : `(X,Y|Z)`

⭐ If you want to developp a new application, hyperspy can support its implementation as a new signal type.

In [ ]:
print(spim)

## I. Metadata

The Signal object has a metadata attribute. Some of it is used for signal specific function (e.g. beam energy). In this tutorial all the relevant metatadata is already filled in.

🏠 You can add new metadata categories this way : `spim.metadata.Category1 = {}` and new metadata entries this way : `spim.metadata.Category1.entry1 = 'entry1 value'`

⭐For some of the proprietary file formats, some of the metadata are automatically read and loaded by hyperspy. Be careful, what is loaded and what is not.

In [ ]:
print(spim.metadata)

## I. Axes manager

The signal object has an axes_manager attribute to manage the calibrations of both the navigation axes and the signal axis. The Graphical User Interface (GUI) enables you to edit the values. 

⭐ Just typing `spim.axes_manager` prints it. You can then acces the values using e.g. : `spim.axes_manager[0].scale` or `spim.axes_manager[axis_name].size`

In [ ]:
spim.axes_manager.gui()

## I. Plotting

For 3 dimensional data, the plotting makes 2 windows pop-up. The first window contains an image in which the intensity of each pixel is the sum of the intensities of the corresponding spectrum (sum over the signal axis). A single pixel Region Of Interest is displayed in the top left of the image. The second window represents the spectrum associated to the ROI. 

You can move this ROI using arrow keys.

In [ ]:
spim.plot()

### I. Advanced Plotting

The code below is a way to improve the basic visualisation of hyperspy. In addition to the single pixel ROI (which cannot be removed), there is an ajustable ROI. A thrid window is also added and displays the sum of spectra in the adjustable ROI.

In [18]:
# Get safe coordinates to initialize the ROI
half_h = spim.axes_manager[0].offset + spim.axes_manager[0].scale*0.5*spim.axes_manager[0].size 
half_v = spim.axes_manager[1].offset + spim.axes_manager[1].scale*0.5*spim.axes_manager[1].size
third_h = spim.axes_manager[0].offset + spim.axes_manager[0].scale*0.333*spim.axes_manager[0].size 
third_v = spim.axes_manager[1].offset + spim.axes_manager[1].scale*0.333*spim.axes_manager[1].size 

# Create the ROI
img_ROI=hs.roi.RectangularROI(third_h,
                              third_v,
                              half_h,
                              half_v)

# Plot the spim as usual
spim.plot()
# Link, interactively, the ROI to the spim
spim_ROI=img_ROI.interactive(spim)

# Compute interactively the sum over the ROI
spectrum=hs.interactive(spim_ROI.sum,
           event=spim_ROI.axes_manager.events.any_axis_changed)
# Plot the computed sum
spectrum.plot(True)

# I. Data Preprocessing

## I. Cropping

The crop function can be used to cut out parts of the data. **Be careful : it overwrites the data.** The syntax is as follows : `spim.crop(axis,start,stop)`. Note that axis is either an integer or an axis name. 

For `start` and `stop`, the options are :
- Integer indices : They do not take into account the calibration and correspond to matrix indices.
- Float indices : They take into account the calibration.

⭐Using `.inav` for navigation axes or `.isig` for signal axes is a more powerful way to slice data. It supports integer indexing, float indices and even string indices with e.g. `spim.inav["10 nm","20 nm"]`. It also supports advanced slicing such as `spim.isig[::2]` or `spim.inav[::,-1]`.



In [ ]:
spim.crop("y",10.0,20.0)

## I. Spikes removal

In [ ]:
spim.spikes_removal_tool()

## I. Energy alignement

In [ ]:
# spim.align_1D

# I. Chemical mapping

## I. Background removal

There is an interactive background removal tool in hyperpsy you can call using : `spim.remove_background()`. Below the cell where the function is executed, there is a "Help" button. It will explain you how to use the tool. **Be careful : Once you click on apply, the spim object is modified.**

For EELS, the Power Law background is the most adapted.

In [26]:
spim.remove_background()

C:\Users\teurtrie\.conda\envs\qem\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## I. Chemical map

**This step should be performed only on background subtracted spim.** 

- The first cell will make the a window pop-up. The window displays the sum of all the spectra of the spectrum image and a selection ROI. Once you're happy with the selection you can execute the next cell.
- Plot the map of integrated counts inside the selected energy range.

You can then go back and forth between the 2 cells below.

In [23]:
# Get safe initial coordinates for the ROI.
half_e = spim.axes_manager[2].offset + spim.axes_manager[2].scale*0.5*spim.axes_manager[2].size
third_e = spim.axes_manager[2].offset + spim.axes_manager[2].scale*0.333*spim.axes_manager[2].size

# Sum over all the spectra of the spim
full_spectrum = spim.integrate1D(axis = (0,1))
# Initialize the ROI
spectrum_ROI = hs.roi.SpanROI(third_e,half_e)

# Plot both the spectrum and the ROI (interactively)
full_spectrum.plot(True)
spectrum_ROI.interactive(full_spectrum)

<EELSSpectrum, title: , dimensions: (|171)>

In [24]:
chem_map = spim.isig[spectrum_ROI.left:spectrum_ROI.right].sum(axis = 2)
chem_map.plot()